In [2]:
import pandas as pd
import os
import numpy as np

In [13]:
dataset = pd.read_csv("US30M5_PivotBuySellProfit.csv", low_memory=False)

In [7]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 24 columns):
DateTime              100000 non-null object
Open                  100000 non-null float64
High                  100000 non-null float64
Low                   100000 non-null float64
Close                 100000 non-null float64
TickVolume            100000 non-null int64
DONOTUSE              100000 non-null int64
Pivot                 100000 non-null int64
PivotType             18979 non-null object
CandleType            100000 non-null object
HigherHigh            100000 non-null int64
ExcessHigh            100000 non-null float64
LowerLow              100000 non-null int64
ExcessLow             100000 non-null float64
BuyWin                100000 non-null int64
SellWin               100000 non-null int64
Outside               100000 non-null int64
Inside                100000 non-null int64
BuyProfit             93209 non-null float64
SellProfit            93209 non-null fl

In [4]:
# generate pivot point flag
# definition of pivot points is:
# high is higher than previous high and next high, low is higher than previous low and next low -> peak
# low is lower than previous low and next low, high is lower than previous high and next high -> trough

# create pivot point array where 0 means not a pivot point and 1 means is a pivot point
pivot = [0]
pivotType = [np.nan]
for i in range(1, len(dataset)-1):
    print(str(i) + "/" + str(len(dataset)-2), end="\r")
    # check whether point is a peak
    if (float(dataset.iloc[i]["High"]) > float(dataset.iloc[i+1]["High"])) and (float(dataset.iloc[i]["High"]) > float(dataset.iloc[i-1]["High"])) and (float(dataset.iloc[i]["Low"]) > float(dataset.iloc[i+1]["Low"])) and (float(dataset.iloc[i]["Low"]) > float(dataset.iloc[i-1]["Low"])):
        pivot.append(1)
        pivotType.append("Peak")
    # check whether point is a trough
    elif (float(dataset.iloc[i]["High"]) < float(dataset.iloc[i+1]["High"])) and (float(dataset.iloc[i]["High"]) < float(dataset.iloc[i-1]["High"])) and (float(dataset.iloc[i]["Low"]) < float(dataset.iloc[i+1]["Low"])) and (float(dataset.iloc[i]["Low"]) < float(dataset.iloc[i-1]["Low"])):
        pivot.append(1)
        pivotType.append("Trough")
    # not a pivot point
    else:
        pivot.append(0)
        pivotType.append(np.nan)
# append the final value as not pivot
pivot.append(0)
pivotType.append(np.nan)

# append pivot and pivotType columns
try:
    dataset.insert(len(dataset.columns), "Pivot", pivot)
    dataset.insert(len(dataset.columns), "PivotType", pivotType)
except ValueError:
    print("Pivot and/or PivotType columns already exist in dataset")

In [5]:
# Generate candle type flag
# Flag is Higher when close is higher than open, Lower when close is lower than open, and Doji when close is equal to open
candleType = []
for i in range(len(dataset)):
    print(str(i+1) + "/" + str(len(dataset)), end="\r")
    if float(dataset.iloc[i]["Close"]) > float(dataset.iloc[i]["Open"]):
        candleType.append("Higher")
    elif float(dataset.iloc[i]["Close"]) < float(dataset.iloc[i]["Open"]):
        candleType.append("Lower")
    else:
        candleType.append("Doji")
        
# append pivot and pivotType columns
try:
    dataset.insert(len(dataset.columns), "CandleType", candleType)
except ValueError:
    print("CandleType column already exist in dataset")

In [14]:
# calculate whether candle high extended higher than previous high. 1 - yes and 0 - no
# calculate whether candle low extended lower than previous low. 1 - yes and 0 - no
# calculate magnitude of excess extension equals to current high - previous high or current low - previous low

higherHigh = [0]
lowerLow = [0]
excessHigh = [0]
excessLow = [0]

for i in range(1, len(dataset)):
    print(str(i) + "/" + str(len(dataset)-1), end="\r")
    excessHigh.append(float(dataset.iloc[i]["High"]) - float(dataset.iloc[i-1]["High"]))
    excessLow.append(float(dataset.iloc[i]["Low"]) - float(dataset.iloc[i-1]["Low"]))
    
    if excessHigh[i] > 0:
        higherHigh.append(1)
    else:
        higherHigh.append(0)
    if excessLow[i] < 0:
        lowerLow.append(1)
    else:
        lowerLow.append(0)
        
# append higherHigh, lowerLog, excessHigh, and excessLow columns
try:
    dataset.insert(len(dataset.columns), "HigherHigh", higherHigh)
    dataset.insert(len(dataset.columns), "ExcessHigh", excessHigh)
    dataset.insert(len(dataset.columns), "LowerLow", lowerLow)
    dataset.insert(len(dataset.columns), "ExcessLow", excessLow)
except ValueError:
    print("HigherHigh and/or ExcessHigh and/or LowerLow and/or ExcessLow already exist in dataset")

In [28]:
# calculate whether candle is an outside or inside candle
# outside candle definition is high higher than previous candle's high and low lower than previous candle's low. 1 for yes and 0 for no
# inside candle definition is high lower than previous candle's high and low higher than previous candle's low 1 for yes and 0 for no

dataset["Outside"] = np.where((dataset["HigherHigh"] == 1) & (dataset["LowerLow"] == 1), 1, 0)
dataset["Inside"] = np.where((dataset["HigherHigh"] == 0) & (dataset["LowerLow"] == 0), 1, 0)

dataset.at[0, "Outside"] = 0
dataset.at[0, "Inside"] = 0

In [34]:
# calculate profit at each point for buy and sell operation
# for buy, criteria for profit taking is if low of current candle exceeds low of previous candle
# for sell, criteria for profit taking is if high of current candle exceeds high of previous candle
# profit taking occurs at 0.1 below previous low for buy, and 0.1 above previous high for sell
# assume buy and at open using limit order at closing price of previous candle
buyProfit = [0]
sellProfit = [0]
exitPriceSell = [0]
exitPriceBuy = [0]

higherHighs = list(dataset.loc[(dataset["HigherHigh"] == 1)].index)
lowerLows = list(dataset.loc[(dataset["LowerLow"] == 1)].index)
higherHighsIndex = 0
lowerLowsIndex = 0
exitPriceHigh = float(dataset.iloc[higherHighs[higherHighsIndex]-1]["High"] + 0.1)
exitPriceLow = float(dataset.iloc[lowerLows[lowerLowsIndex]-1]["Low"] - 0.1)

for i in range(1, len(dataset)-1):
    print(str(i) + "/" + str(len(dataset)-2), end="\r")
    
    # initialise
    currentDateTime = str(dataset.iloc[i]["DateTime"])
    currentLow = float(dataset.iloc[i]["Low"])
    currentHigh = float(dataset.iloc[i]["High"])
    previousClose = float(dataset.iloc[i-1]["Close"])
    if i > higherHighs[higherHighsIndex]:
        higherHighsIndex += 1
        try:
            # prioritise open price
            if dataset.iloc[higherHighs[higherHighsIndex]]["Open"] > dataset.iloc[higherHighs[higherHighsIndex]-1]["High"]:
                exitPriceHigh = dataset.iloc[higherHighs[higherHighsIndex]]["Open"]
            else:
                exitPriceHigh = float(dataset.iloc[higherHighs[higherHighsIndex]-1]["High"] + 0.1)
        except IndexError:
            exitPriceHigh = float(dataset.iloc[i+1]["Close"])
        
    if i > lowerLows[lowerLowsIndex]:
        lowerLowsIndex += 1
        try:
            if dataset.iloc[lowerLows[lowerLowsIndex]]["Open"] < dataset.iloc[lowerLows[lowerLowsIndex]-1]["Low"]:
                exitPriceLow = dataset.iloc[lowerLows[lowerLowsIndex]]["Open"]
            else:
                exitPriceLow = float(dataset.iloc[lowerLows[lowerLowsIndex]-1]["Low"] - 0.1)
        except IndexError:
            exitPriceLow = float(dataset.iloc[i+1]["Close"])
    
    # calculate buy
    buyPrice = previousClose if ((currentLow <= previousClose) and (previousClose <= currentHigh)) else np.nan
    
    # calculate buy profit
    buyProfit.append((exitPriceLow - buyPrice) if buyPrice else np.nan)
    
    # calculate sell
    sellPrice = previousClose if ((currentLow <= previousClose) and (previousClose <= currentHigh)) else np.nan
    
    # calculate sell profit
    sellProfit.append((sellPrice - exitPriceHigh) if sellPrice else np.nan)
    
    # append sell exit price
    exitPriceSell.append(exitPriceHigh)
    
    # append buy exit price
    exitPriceBuy.append(exitPriceLow)
    
buyProfit.append(0)
sellProfit.append(0)
exitPriceSell.append(0)
exitPriceBuy.append(0)

# append buy and sell profit columns
try:
    dataset.insert(len(dataset.columns), "BuyProfit", buyProfit)
    dataset.insert(len(dataset.columns), "SellProfit", sellProfit)
    dataset.insert(len(dataset.columns), "ExitPriceBuy", exitPriceBuy)
    dataset.insert(len(dataset.columns), "ExitPriceSell", exitPriceSell)
except ValueError:
    print("BuyProfit/SellProfit/ExitPriceBuy/ExitPriceSell columns already exist in dataset")

In [35]:
dataset.drop(labels = ["PivotBuySell", "PivotBuySellProfit"], axis = "columns", inplace = True)

In [ ]:
# calculate profit at each point for buy and sell operation (for outside/inside candle calculations)
# for buy, criteria for profit taking is if low of current candle exceeds low of previous candle
# for sell, criteria for profit taking is if high of current candle exceeds high of previous candle
# profit taking occurs at 0.1 below previous low for buy, and 0.1 above previous high for sell
# assume buy and at open using limit order at closing price of previous candle

buyHigherHighProfit = [0]
sellLowerLowProfit = [0]

higherHighs = list(dataset.loc[(dataset["HigherHigh"] == 1)].index)
lowerLows = list(dataset.loc[(dataset["LowerLow"] == 1)].index)
higherHighsIndex = 0
lowerLowsIndex = 0
exitPriceHigh = float(dataset.iloc[higherHighs[higherHighsIndex]-1]["High"] + 0.1)
exitPriceLow = float(dataset.iloc[lowerLows[lowerLowsIndex]-1]["Low"] - 0.1)

for i in range(1, len(dataset)-1):
    print(str(i) + "/" + str(len(dataset)-2), end="\r")
    
    # initialise
    currentDateTime = str(dataset.iloc[i]["DateTime"])
    currentOpen = float(dataset.iloc[i]["Low"])
    currentHigh = float(dataset.iloc[i]["High"])
    previousHigh = float(dataset.iloc[i-1]["High"])
    previousLow = float(dataset.iloc[i-1]["Low"])
    if i > higherHighs[higherHighsIndex]:
        higherHighsIndex += 1
        try:
            exitPriceHigh = float(dataset.iloc[higherHighs[higherHighsIndex]-1]["High"] + 0.1)
        except IndexError:
            exitPriceHigh = float(dataset.iloc[i+1]["Close"])
        
    if i > lowerLows[lowerLowsIndex]:
        lowerLowsIndex += 1
        try:
            exitPriceLow = float(dataset.iloc[lowerLows[lowerLowsIndex]-1]["Low"] - 0.1)
        except IndexError:
            exitPriceLow = float(dataset.iloc[i+1]["Close"])
    
    # calculate buy only if current candle is higher high and not lower low
    if dataset.iloc[i]["HigherHigh"] and not dataset.iloc[i]["LowerLow"]:
        # determine whether to take open price or previous high + 0.1
        buyPrice = currentOpen if (currentOpen > previousHigh) else previousHigh + 0.1
    else:
        buyPrice = np.nan
    
    # calculate buy profit
    buyProfit.append((exitPriceLow - buyPrice) if buyPrice else np.nan)
    
    # calculate sell only if current candle is lower low and not higher high
    sellPrice = previousClose if ((currentLow <= previousClose) and (previousClose <= currentHigh)) else np.nan
    
    # calculate sell profit
    sellProfit.append((sellPrice - exitPriceHigh) if sellPrice else np.nan)
    
buyProfit.append(0)
sellProfit.append(0)

# append buy and sell profit columns
try:
    dataset.insert(len(dataset.columns), "BuyProfit", buyProfit)
    dataset.insert(len(dataset.columns), "SellProfit", sellProfit)
except ValueError:
    print("BuyProfit and/or SellProfit columns already exist in dataset")

In [6]:
# generate dataset of horizontal support and resistance lines
# dataset includes start datetime and price level at each horizontal line, as well as number of rejections from each line
# a horizontal trendline is defined as the price level at a pivot point
# a rejection is defined as either the low reaching lower than horizontal price level, and close is higher than horizontal price level
# or high reaching higher than horizontal price level, and close is lower than horizontal price level
# price level of each horizontal line is the high of a pivot point if pivot point type = 'Peak', and low of a pivot point if pivot point type = 'Trough'
# failed rejections are all horizontal price levels in between open and close inclusive

# create all possible horizontal support and resistance lines
horizontalLines = dataset.loc[dataset["Pivot"] == 1][["DateTime", "PivotType", "High", "Low"]].reset_index(drop=True)
horizontalLines["PriceLevel"] = np.where(horizontalLines["PivotType"] == "Peak", horizontalLines["High"], horizontalLines["Low"])

# drop columns
horizontalLines.drop(labels=["PivotType", "High", "Low"], axis="columns", inplace=True)

RejectionCount = pd.Series([0 for i in range(len(horizontalLines))])
SupportRejectionCount = pd.Series([0 for i in range(len(horizontalLines))]) # candle has low lower than or equal to horizontal price level, and close higher than horizontal price level
ResistanceRejectionCount = pd.Series([0 for i in range(len(horizontalLines))]) # candle has high higher than or equal to horizontal price level, and close lower than horizontal price level
RejectionFailedCount = pd.Series([0 for i in range(len(horizontalLines))])
startIndex = int(dataset.loc[dataset['DateTime'] == str(horizontalLines.iloc[0]['DateTime'])].index[0]) + 1

# calculate rejections for each horizontal line using entire candle chart dataset
for i in range(startIndex, len(dataset)):
    print(str(i+1) + "/" + str(len(dataset)), end="\r")
    currentTime = str(dataset.loc[i]["DateTime"])
    
    # calculate resistance rejections
    rangeHigh = float(dataset.loc[i]["High"])
    rangeLow = max(float(dataset.loc[i]["Close"]), float(dataset.loc[i]["Open"])) # take max to only obtain higher part of candle
    # indices of horizontal lines where resistance rejection occurs
    indices = horizontalLines.loc[(horizontalLines["PriceLevel"] <= rangeHigh) & (horizontalLines["PriceLevel"] > rangeLow) & (horizontalLines["DateTime"] < currentTime)].index
    # increment rejection count
    RejectionCount.iloc[indices] += 1
    ResistanceRejectionCount.iloc[indices] += 1
    
    # calculate support rejections
    rangeHigh = min(float(dataset.loc[i]["Close"]), float(dataset.loc[i]["Open"])) # take min to only obtain lower part of candle
    rangeLow = float(dataset.loc[i]["Low"])
    # indices of horizontal lines where support rejection occurs
    indices = horizontalLines.loc[(horizontalLines["PriceLevel"] >= rangeLow) & (horizontalLines["PriceLevel"] < rangeHigh) & (horizontalLines["DateTime"] < currentTime)].index
    # increment rejection count
    RejectionCount.iloc[indices] += 1
    SupportRejectionCount.iloc[indices] += 1
    
    # calculate failed rejections
    rangeHigh = max(float(dataset.loc[i]["Close"]), float(dataset.loc[i]["Open"]))
    rangeLow = min(float(dataset.loc[i]["Close"]), float(dataset.loc[i]["Open"]))
    # indices of horizontal lines where failed rejection occurs
    indices = horizontalLines.loc[(horizontalLines["PriceLevel"] <= rangeHigh) & (horizontalLines["PriceLevel"] >= rangeLow) & (horizontalLines["DateTime"] < currentTime)].index
    RejectionFailedCount.iloc[indices] += 1
    
    
try:
    horizontalLines.insert(len(horizontalLines.columns), "RejectionCount", RejectionCount)
    horizontalLines.insert(len(horizontalLines.columns), "SupportRejectionCount", SupportRejectionCount)
    horizontalLines.insert(len(horizontalLines.columns), "ResistanceRejectionCount", ResistanceRejectionCount)
    horizontalLines.insert(len(horizontalLines.columns), "RejectionFailedCount", RejectionFailedCount)
except ValueError:
    print("RejectionCount and/or SupportRejectionCount and/or ResistanceRejectionCount and/or RejectionFailedCount already exists in table horizontalLines")

In [32]:
dataset

,DateTime,Open,High,Low,Close,TickVolume,DONOTUSE,Pivot,PivotType,CandleType,...,BuyWin,SellWin,Outside,Inside,BuyProfit,SellProfit,PivotBuySell,PivotBuySellProfit,ExitPriceBuy,ExitPriceSell
0,2019.10.08 01:00,26459.4,26464.4,26448.4,26456.4,261,0,0,NaN,Lower,...,0,0,0,0,0.0,0.0,No trade,0.0,0.0,0.0
1,2019.10.08 01:05,26456.9,26461.4,26453.9,26460.4,78,0,0,NaN,Higher,...,1,0,0,1,13.9,-5.1,No trade,0.0,26470.3,26461.5
2,2019.10.08 01:10,26460.9,26471.4,26460.4,26469.9,105,0,0,NaN,Higher,...,1,0,0,0,9.9,-1.1,No trade,0.0,26470.3,26461.5
3,2019.10.08 01:15,26470.4,26473.9,26468.4,26471.4,82,0,0,NaN,Higher,...,1,0,0,0,0.4,-1.6,No trade,0.0,26470.3,26471.5
4,2019.10.08 01:20,26470.9,26476.9,26470.4,26476.4,56,0,0,NaN,Higher,...,0,0,0,0,-1.1,-2.6,No trade,0.0,26470.3,26474.0
5,2019.10.08 01:25,26475.9,26477.4,26465.4,26466.4,47,0,0,NaN,Lower,...,0,0,1,0,-6.1,-0.6,No trade,0.0,26470.3,26477.0
6,2019.10.08 01:30,26467.4,26474.9,26467.4,26474.9,66,0,0,NaN,Higher,...,0,0,0,1,NaN,NaN,No trade,0.0,26478.3,26475.0
7,2019.10.08 01:35,26474.4,26482.4,26474.4,26482.4,85,0,0,NaN,Higher,...,1,0,0,0,3.4,-0.1,No trade,0.0,26478.3,26475.0
8,2019.10.08 01:40,26483.9,26484.4,26475.9,26478.4,62,0,0,NaN,Lower,...,0,0,0,0,-4.1,-0.1,No trade,0.0,26478.3,26482.5
9,2019.10.08 01:45,26478.9,26482.9,26478.4,26480.4,55,0,0,NaN,Higher,...,0,1,0,1,-0.1,6.4,No trade,0.0,26478.3,26472.0


In [36]:
# define first strategy - PivotBuySell
# When most recent pivot is trough, only buy. When most recent pivot is peak, only sell

try:
    pivotPointIndex = list(dataset.loc[dataset["Pivot"]==1].index)
    nextPivotPointIndexIndex = 0
    startIndex = pivotPointIndex[nextPivotPointIndexIndex] + 2
    pivotBuySell = ["No trade" for i in range(startIndex)]
    pivotBuySellProfit = [0.0 for i in range(startIndex)]
    
    for i in range(startIndex, len(dataset)):
        print(str(i+1) + "/" + str(len(dataset)), end="\r")
        if (i-2) == pivotPointIndex[nextPivotPointIndexIndex]:
            pivotTypeOfInterest = str(dataset.iloc[pivotPointIndex[nextPivotPointIndexIndex]]["PivotType"])
            nextPivotPointIndexIndex += 1
        
        if pivotTypeOfInterest == "Peak":
            pivotBuySell.append("Sell")
            pivotBuySellProfit.append(dataset.iloc[i]["SellProfit"])
        elif pivotTypeOfInterest == "Trough":
            pivotBuySell.append("Buy")
            pivotBuySellProfit.append(dataset.iloc[i]["BuyProfit"])
            
    dataset.insert(len(dataset.columns), "PivotBuySell", pivotBuySell)
    dataset.insert(len(dataset.columns), "PivotBuySellProfit", pivotBuySellProfit)
            
except Exception as e:
    print(e)

In [24]:
# from eyeballing trades that fail and the events occuring before them
# suspect that the major reason for failure is due to inside or outside bar which causes a direction change in price
# but not picked up by the current definition of a pivot point
# investigate this by obtaining list of continuous loss

index = list(dataset.loc[(dataset["PivotBuySellProfit"] < 0) | (dataset["PivotBuySellProfit"].isna())].index)
currentIndex = index[0]
startIndex = []
tentativeStartIndex = currentIndex
firstInsideCandlePositionInSequence = []
firstInsideCandleIndex = -1
firstOutsideCandlePositionInSequence = []
firstOutsideCandleIndex = -1
sequenceLength = []
minInsideOutsidePosition = []
endIndex = []

for i in range(1, len(index)):
    print(str(i+1) + "/" + str(len(index)), end="\r")
    if (dataset.iloc[currentIndex]["Outside"] == 1) and (firstOutsideCandleIndex == -1):
        firstOutsideCandleIndex = currentIndex
    elif (dataset.iloc[currentIndex]["Inside"] == 1) and (firstInsideCandleIndex == -1):
        firstInsideCandleIndex = currentIndex
    nextIndex = index[i]
    if currentIndex < nextIndex - 1:
        if tentativeStartIndex < currentIndex:
            startIndex.append(tentativeStartIndex)
            endIndex.append(currentIndex)
            firstInsideCandlePositionInSequence.append(-1 if (firstInsideCandleIndex == -1) else (firstInsideCandleIndex - tentativeStartIndex + 1))
            firstOutsideCandlePositionInSequence.append(-1 if (firstOutsideCandleIndex == -1) else (firstOutsideCandleIndex - tentativeStartIndex + 1))
            sequenceLength.append(currentIndex - tentativeStartIndex + 1)
            minInsideOutsidePosition.append(min(firstOutsideCandlePositionInSequence[len(firstOutsideCandlePositionInSequence)-1] if (firstInsideCandlePositionInSequence[len(firstInsideCandlePositionInSequence)-1] == -1) else firstInsideCandlePositionInSequence[len(firstInsideCandlePositionInSequence)-1]
                                                ,firstInsideCandlePositionInSequence[len(firstInsideCandlePositionInSequence)-1] if (firstOutsideCandlePositionInSequence[len(firstOutsideCandlePositionInSequence)-1] == -1) else firstOutsideCandlePositionInSequence[len(firstOutsideCandlePositionInSequence)-1]))
        tentativeStartIndex = nextIndex
        firstOutsideCandleIndex = -1
        firstInsideCandleIndex = -1
    currentIndex = nextIndex
    
dataDict = {"StartIndex": startIndex, "EndIndex": endIndex, "SequenceLength": sequenceLength, "FirstInsideCandle": firstInsideCandlePositionInSequence, "FirstOutsideCandle": firstOutsideCandlePositionInSequence, "MinOutsideInsidePosition": minInsideOutsidePosition}
LossSequence = pd.DataFrame(data=dataDict, dtype=pd.Int64Dtype)

In [25]:
LossSequence.to_csv("LossSequencePivotBuySell.csv", index=False)

In [ ]:
# 

In [14]:
# define second strategy - PivotBuySell_v2
# Determine the most recent point of interest (peak, trough, outside candle)
# When most recent pivot is trough, only buy. When most recent pivot is peak, only sell
# When encountering outisde candle, make no trade until 2 criteria are satisfied for both buy and sell:
# Buy:
# - first candle higher than high of outside candle
# - next candle higher than high of first candle
# - buy at second candle at first point higher than high of first candle, with open price taking higher priority, otherwise previous high + 0.1
# Sell:
# - first candle lower than low of outside candle
# - next candle lower than low of first candle
# - sell at second candle at first point lower than low of first candle, with open price taking higher priority, otherwise previous low - 0.1
# When encountering inside candle, treat previous candle as outside candle

try:
    outsideInsidePointIndex = list(dataset.loc[(dataset["Outside"]==1) | (dataset["Inside"]==1)].index)
    nextOutsideInsidePointIndexIndex = 0
    pivotPointIndex = list(dataset.loc[dataset["Pivot"]==1].index)
    nextPivotPointIndexIndex = 0
    startIndex = min(pivotPointIndex[nextPivotPointIndexIndex], outsideInsidePointIndex[nextOutsideInsidePointIndexIndex]) + 1
    # determine start index as the first index after an outside/inside or 2 indices after a pivot depending on whether first outside/inside index is before first pivot
    if outsideInsidePointIndex[nextOutsideInsidePointIndexIndex] < pivotPointIndex[nextPivotPointIndexIndex]:
        startIndex = outsideInsidePointIndex[nextOutsideInsidePointIndexIndex] + 1
        if dataset.iloc[outsideInsidePointIndex[nextOutsideInsidePointIndexIndex]]["Inside"]:
            # initialise values for Inside candle
            mostRecentPointType = "Inside"
            OutsideHigh = dataset.iloc[outsideInsidePointIndex[nextOutsideInsidePointIndexIndex]-1]["High"]
            OutsideLow = dataset.iloc[outsideInsidePointIndex[nextOutsideInsidePointIndexIndex]-1]["Low"]
        else:
            # initialise values for Outside candle
            mostRecentPointType = "Outside"
            OutsideHigh = dataset.iloc[outsideInsidePointIndex[nextOutsideInsidePointIndexIndex]]["High"]
            OutsideLow = dataset.iloc[outsideInsidePointIndex[nextOutsideInsidePointIndexIndex]]["Low"]
    else:
        # initialise values for Pivot candle
        startIndex = pivotPointIndex[nextPivotPointIndexIndex] + 2
        mostRecentPointType = "Pivot"
        OutsideHigh = -1
        OutsideLow = -1
    pivotBuySell = ["No trade" for i in range(startIndex)]
    pivotBuySellProfit = [0.0 for i in range(startIndex)]
    # initialise variables to calculate for consecutive higher highs or consecutive lower lows. Start counting at first higher high higher than OutsideHigh or lower low lower than OutsideLow
    followThroughHighCount = 0
    followThroughLowCount = 0

    for i in range(startIndex, len(dataset)):
        print(str(i+1) + "/" + str(len(dataset)), end="\r")

        # determine whether most recent point is (Pivot, Inside, Outside)
        if (i-2) == pivotPointIndex[nextPivotPointIndexIndex]:
            # most recent point is Pivot
            mostRecentPointType = "Pivot"
            pivotTypeOfInterest = str(dataset.iloc[pivotPointIndex[nextPivotPointIndexIndex]]["PivotType"])
            nextPivotPointIndexIndex += 1
        elif (i-1) == outsideInsidePointIndex[nextOutsideInsidePointIndexIndex]:
            # most recent point is Outside/Inside
            if dataset.iloc[outsideInsidePointIndex[nextOutsideInsidePointIndexIndex]]["Inside"]:
                mostRecentPointType = "Inside"
                OutsideHigh = dataset.iloc[outsideInsidePointIndex[nextOutsideInsidePointIndexIndex]-1]["High"]
                OutsideLow = dataset.iloc[outsideInsidePointIndex[nextOutsideInsidePointIndexIndex]-1]["Low"]
            else:
                mostRecentPointType = "Outside"
                OutsideHigh = dataset.iloc[outsideInsidePointIndex[nextOutsideInsidePointIndexIndex]]["High"]
                OutsideLow = dataset.iloc[outsideInsidePointIndex[nextOutsideInsidePointIndexIndex]]["Low"]
            # reset follow through counts
            followThroughHighCount = 0
            followThroughLowCount = 0
            # increment index
            nextOutsideInsidePointIndexIndex += 1

        if mostRecentPointType == "Pivot":
            if pivotTypeOfInterest == "Peak":
                pivotBuySell.append("Sell")
                pivotBuySellProfit.append(dataset.iloc[i]["SellProfit"])
            elif pivotTypeOfInterest == "Trough":
                pivotBuySell.append("Buy")
                pivotBuySellProfit.append(dataset.iloc[i]["BuyProfit"])
        else:
            # determine whether high higher than OutsideHigh and is higher high or low lower than OutsideLow and is lower low. Reset if follow through is broken
            if (dataset.iloc[i]["High"] > OutsideHigh) and (dataset.iloc[i]["HigherHigh"]==1):
                # reset count for low
                followThroughLowCount = 0
                # increment follow through for high
                followThroughHighCount += 1
            elif (dataset.iloc[i]["Low"] < OutsideLow) and (dataset.iloc[i]["LowerLow"]==1):
                # reset count for high
                followThroughHighCount = 0
                # increment follow through for low
                followThroughLowCount += 1
            else:
                # reset if not reached higher or lower than outside bar, or not reach higher or lower than previous bar
                followThroughHighCount = 0
                followThroughLowCount = 0

            # buy or sell when follow through count = 2
            if followThroughHighCount == 2:
                # buy at higher than previous high + 0.1 only if price comes down to it
                currentLow = dataset.iloc[i]["Low"]
                previousHigh = dataset.iloc[i-1]["High"]
                exitPriceLow = dataset.iloc[i]["ExitPriceBuy"]
                pivotBuySell.append("Buy")
                # calculate buy
                buyPrice = (previousHigh + 0.1) if (currentLow <= (previousHigh + 0.1)) else np.nan
                pivotBuySellProfit.append((exitPriceLow - buyPrice) if buyPrice else np.nan)
                # treat subsequent bars as if prior bar is trough pivot
                mostRecentPointType = "Pivot"
                pivotTypeOfInterest = "Trough"
            elif followThroughLowCount == 2:
                # sell at lower than previous low - 0.1 only if price comes down to it
                currentHigh = dataset.iloc[i]["High"]
                previousLow = dataset.iloc[i-1]["Low"]
                exitPriceHigh = dataset.iloc[i]["ExitPriceSell"]
                pivotBuySell.append("Sell")
                # calculate sell
                sellPrice = (previousLow - 0.1) if (currentHigh >= (previousLow - 0.1)) else np.nan
                pivotBuySellProfit.append((sellPrice - exitPriceHigh) if sellPrice else np.nan)
                # treat subsequent bars as if prior bar is peak pivot
                mostRecentPointType = "Pivot"
                pivotTypeOfInterest = "Peak"
            else:
                # no trade
                pivotBuySell.append("No trade")
                pivotBuySellProfit.append(0)

    dataset.insert(len(dataset.columns), "PivotBuySell_v2", pivotBuySell)
    dataset.insert(len(dataset.columns), "PivotBuySellProfit_v2", pivotBuySellProfit)
            
except Exception as e:
    print(e)

In [8]:
horizontalLines.to_csv("HorizontalSupportResistance.csv", index=False)

In [61]:
dataset.drop(labels=["CandleType"], axis="columns", inplace=True)

In [22]:
# calculate buy and sell populations of winners and losers
dataset["BuyWin"] = np.where(dataset["BuyProfit"] > 0, 1, 0)
dataset["SellWin"] = np.where(dataset["SellProfit"] > 0, 1, 0)

In [15]:
dataset.to_csv("US30M5_PivotBuySellProfit_v2.csv", index=False)

In [67]:
RejectionCount = pd.Series([0 for i in range(len(horizontalLines))])

In [28]:
dataDict = {"StartIndex": startIndex, "EndIndex": endIndex, "SequenceLength": sequenceLength, "FirstInsideCandle": firstInsideCandlePositionInSequence, "FirstOutsideCandle": firstOutsideCandlePositionInSequence}
LossSequence = pd.DataFrame(data=dataDict, dtype=pd.Int64Dtype)

In [40]:
list(dataset.loc[dataset["PivotBuySellProfit"] < 0].index)

[15,
 16,
 17,
 19,
 20,
 21,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 49,
 50,
 51,
 52,
 53,
 55,
 56,
 57,
 58,
 59,
 64,
 65,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 85,
 86,
 89,
 90,
 91,
 92,
 93,
 95,
 96,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 111,
 112,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 129,
 130,
 131,
 132,
 133,
 134,
 146,
 148,
 149,
 150,
 151,
 152,
 153,
 155,
 156,
 159,
 160,
 161,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 177,
 178,
 179,
 180,
 181,
 182,
 186,
 187,
 188,
 189,
 190,
 193,
 194,
 195,
 196,
 197,
 198,
 199,
 200,
 201,
 202,
 203,
 204,
 205,
 207,
 208,
 211,
 212,
 213,
 214,
 215,
 216,
 217,
 218,
 219,
 220,
 221,
 222,
 223,
 224,
 226,
 227,
 228,
 229,
 230,
 234,
 235,
 236,
 237,
 238,
 240,
 241,
 242,
 243,
 244,
 250,
 251,
 254,
 255,
 256,
 257,
 258,
 25

In [11]:
dataset.loc[dataset["PivotBuySellProfit"] < 0][["CandleType", "Outside", "Inside", "PivotType", "PivotBuySell", "PivotBuySellProfit"]]

,CandleType,Outside,Inside,PivotType,PivotBuySell,PivotBuySellProfit
15,Higher,0,0,Peak,Buy,-1.1
16,Lower,0,0,NaN,Buy,-4.1
17,Higher,0,0,NaN,Sell,-0.1
19,Higher,0,1,NaN,Sell,-5.1
20,Higher,0,0,NaN,Sell,-2.1
21,Higher,0,0,Peak,Sell,-0.1
23,Lower,0,0,NaN,Sell,-4.1
24,Higher,1,0,NaN,Sell,-6.1
25,Higher,0,0,NaN,Sell,-1.6
26,Lower,0,1,NaN,Sell,-7.1


In [16]:
a = np.nan
(a is np.nan)

True

In [19]:
1 if (a is np.nan) else 0

1